# Machine Learning & PyTorch Basic    
* 작성자 : 김가윤   

## Tensor Manipulation

### View (Reshape in Numpy)

In [1]:
import numpy as np
import torch

In [4]:
t = np.array([[[0,1,2],
              [3,4,5]],
              [[6,7,8],
              [9,10,11]]])
ft = torch.FloatTensor(t)
print(ft.shape) # 가로 2 세로 3 높이 2

torch.Size([2, 2, 3])


view를 사용해 모양을 바꾼다. 앞은 모르겠고(-1) 뒤에 두 개 차원, 두 번째 차원은 3개 element를 가지겠다. 원하는 형태로 바꿀 수 있다.

In [5]:
print(ft.view([-1,3]))
print(ft.view([-1,3]).shape)

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])
torch.Size([4, 3])


### Squeeze   
view와 같지만 자동으로 원하는 dimension이 1이면 자동으로 없애주는 역할   
cf. <-> Unsqueeze

In [6]:
ft = torch.FloatTensor([[0],[1],[2]])
print(ft)
print(ft.shape)
print(ft.squeeze())
print(ft.squeeze().shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])
tensor([0., 1., 2.])
torch.Size([3])


### Type Casting   
Change the Tensor type

In [10]:
lt = torch.LongTensor([1,2,3,4])
print(lt.dtype)
print(lt.float())

torch.int64
tensor([1., 2., 3., 4.])


### Concat

In [11]:
x = torch.FloatTensor([[1,2],[3,4]])
y = torch.FloatTensor([[5,6],[7,8]])

In [12]:
print(torch.cat([x,y], dim=0))
print(torch.cat([x,y], dim=1)) # dim 1이 늘어난다.

tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])
tensor([[1., 2., 5., 6.],
        [3., 4., 7., 8.]])


### Stacking

In [13]:
x = torch.FloatTensor([1,2])
y = torch.FloatTensor([5,6])
z = torch.FloatTensor([7,8])

In [14]:
print(torch.stack([x,y,z])) # (3,2)
print(torch.stack([x,y,z], dim=1)) # (2,3)

tensor([[1., 2.],
        [5., 6.],
        [7., 8.]])
tensor([[1., 5., 7.],
        [2., 6., 8.]])


### Ones and Zeros

In [15]:
print(torch.ones_like(x))
print(torch.zeros_like(x))

tensor([1., 1.])
tensor([0., 0.])


### In place Operation

In [16]:
x = torch.FloatTensor([[1,2],[3,4]])
print(x.mul(2.)) # 곱하기, temp memory
print(x)
print(x.mul_(2.)) # 기존 텐서에 적용
print(x)

tensor([[2., 4.],
        [6., 8.]])
tensor([[1., 2.],
        [3., 4.]])
tensor([[2., 4.],
        [6., 8.]])
tensor([[2., 4.],
        [6., 8.]])


## Linear Regression

In [ ]:
## L